# Methods of Reading Large Dataset

 For this task I compared the runtime of excuting `read_csv()`, `read_parquet()`, and `groupby()` using the following methods: 
 
 1. the traditional `pandas`.
 2. the traditional `pandas` with chuncks.
 3. the `dask`.
 4. the `modin` on `ray`.
 5. the `pyspark`.

In [1]:
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd
import ray
import modin.pandas as md
from pyspark.sql import SparkSession
import pyspark.sql.functions as PysparkFunc
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
from pyarrow import csv, parquet

In [2]:
# to use in ray and pyspark
num_cpus = os.cpu_count()
num_cpus

16

In [3]:
data_path = 'data/input/'

In [4]:
csvfile = data_path+'bikes-1.csv'

In [5]:
# print the file size in GB
file_size = os.path.getsize(csvfile)
file_size /= 2**30
file_size = np.round(file_size,2)
print(f'File size is {file_size} GB')

File size is 4.73 GB


In [6]:
%%time
# print number of lines in the file
num_lines = 0
with open(csvfile,'r') as file:
    for line in file:
        num_lines += 1
    print(f'{num_lines}')

38215561
Wall time: 14.2 s


In [7]:
# read the first n lines in the file
n = 10
with open(csvfile,'r') as file:
    for _ in range(n):
        print([file.readline()])

['rental_id,duration,bike_id,end_rental_date_time,end_station_id,end_station_name,start_rental_date_time,start_station_id,start_station_name\n']
['61343322,60.0,12871.0,2016-12-28 00:01:00,660.0,"West Kensington Station, West Kensington",2016-12-28 00:00:00,633,"Vereker Road North, West Kensington"\n']
['61343321,300.0,2837.0,2016-12-28 00:05:00,763.0,"Mile End Park Leisure Centre, Mile End",2016-12-28 00:00:00,531,"Twig Folly Bridge, Mile End"\n']
['61343323,360.0,1269.0,2016-12-28 00:06:00,99.0,"Old Quebec Street, Marylebone",2016-12-28 00:00:00,116,"Little Argyll Street, West End"\n']
['61343325,1140.0,4208.0,2016-12-28 00:20:00,468.0,"Cantrell Road, Bow",2016-12-28 00:01:00,443,"Philpot Street, Whitechapel"\n']
['61343324,,1406.0,,,,2016-12-28 00:01:00,319,"Baldwin Street, St. Luke\'s"\n']
['61343326,1380.0,11485.0,2016-12-28 00:25:00,219.0,"Bramham Gardens, Earl\'s Court",2016-12-28 00:02:00,116,"Little Argyll Street, West End"\n']
['61343327,960.0,7686.0,2016-12-28 00:20:00,710.0

## 1. Traditional `pandas` with `csv` file

In [8]:
%%time
# traditional pandas
# read file
df = None
df = pd.read_csv(csvfile)

Wall time: 41.2 s


In [9]:
df.shape

(38215560, 9)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38215560 entries, 0 to 38215559
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   rental_id               int64  
 1   duration                float64
 2   bike_id                 float64
 3   end_rental_date_time    object 
 4   end_station_id          float64
 5   end_station_name        object 
 6   start_rental_date_time  object 
 7   start_station_id        int64  
 8   start_station_name      object 
dtypes: float64(3), int64(2), object(4)
memory usage: 2.6+ GB


In [11]:
df.isnull().sum()

rental_id                     0
duration                  67617
bike_id                      17
end_rental_date_time      67617
end_station_id            68213
end_station_name          68213
start_rental_date_time        0
start_station_id              0
start_station_name            0
dtype: int64

In [12]:
df['rental_id'].is_unique

True

In [13]:
pd.concat([df.head(),df.tail()])

,rental_id,duration,bike_id,end_rental_date_time,end_station_id,end_station_name,start_rental_date_time,start_station_id,start_station_name
0,61343322,60.0,12871.0,2016-12-28 00:01:00,660.0,"West Kensington Station, West Kensington",2016-12-28 00:00:00,633,"Vereker Road North, West Kensington"
1,61343321,300.0,2837.0,2016-12-28 00:05:00,763.0,"Mile End Park Leisure Centre, Mile End",2016-12-28 00:00:00,531,"Twig Folly Bridge, Mile End"
2,61343323,360.0,1269.0,2016-12-28 00:06:00,99.0,"Old Quebec Street, Marylebone",2016-12-28 00:00:00,116,"Little Argyll Street, West End"
3,61343325,1140.0,4208.0,2016-12-28 00:20:00,468.0,"Cantrell Road, Bow",2016-12-28 00:01:00,443,"Philpot Street, Whitechapel"
4,61343324,NaN,1406.0,NaN,NaN,NaN,2016-12-28 00:01:00,319,"Baldwin Street, St. Luke's"
38215555,101367955,240.0,15436.0,2020-09-01 23:53:00,652.0,"Evesham Street, Avondale",2020-09-01 23:49:00,606,"Addison Road, Holland Park"
38215556,101367958,60.0,10896.0,2020-09-01 23:51:00,488.0,"Reardon Street, Wapping",2020-09-01 23:50:00,458,"Wapping Lane, Wapping"
38215557,101367970,480.0,18116.0,2020-09-01 23:59:00,442.0,"Walmer Road, Avondale",2020-09-01 23:51:00,442,"Walmer Road, Avondale"
38215558,101367999,300.0,16371.0,2020-09-01 23:59:00,511.0,"Sutton Street, Shadwell",2020-09-01 23:54:00,202,"Leman Street, Aldgate"
38215559,101367995,240.0,8408.0,2020-09-01 23:58:00,56.0,"Paddington Street, Marylebone",2020-09-01 23:54:00,367,"Harrowby Street, Marylebone"


In [14]:
%%time
# traditional pandas
# group by
df = None
df = pd.read_csv(csvfile)
grouped_df = df.groupby(['bike_id'], as_index=False).agg({'duration':'sum','rental_id':'count'})

Wall time: 40.9 s


In [15]:
grouped_df.shape

(16019, 3)

In [16]:
pd.concat([grouped_df.head(),grouped_df.tail()])

,bike_id,duration,rental_id
0,1.0,3889465.0,2653
1,2.0,2056260.0,1569
2,4.0,4086285.0,2905
3,5.0,3847900.0,2665
4,6.0,3776526.0,3092
16014,18499.0,6360.0,7
16015,18502.0,6960.0,8
16016,18505.0,4560.0,4
16017,18510.0,19440.0,14
16018,18512.0,8160.0,9


## 2. Traditional `pandas` in chuncks  with `csv` file

In [17]:
num_chunks = 5
chunksize = int(np.ceil(num_lines/num_chunks))
chunksize

7643113

In [18]:
%%time
# pandas chuncks
# read file
iterator = pd.read_csv(csvfile, chunksize=chunksize) 
df = pd.concat([chunck for chunck in iterator]) # time-consuming

Wall time: 40 s


In [19]:
df.shape

(38215560, 9)

In [20]:
pd.concat([df.head(),df.tail()])

,rental_id,duration,bike_id,end_rental_date_time,end_station_id,end_station_name,start_rental_date_time,start_station_id,start_station_name
0,61343322,60.0,12871.0,2016-12-28 00:01:00,660.0,"West Kensington Station, West Kensington",2016-12-28 00:00:00,633,"Vereker Road North, West Kensington"
1,61343321,300.0,2837.0,2016-12-28 00:05:00,763.0,"Mile End Park Leisure Centre, Mile End",2016-12-28 00:00:00,531,"Twig Folly Bridge, Mile End"
2,61343323,360.0,1269.0,2016-12-28 00:06:00,99.0,"Old Quebec Street, Marylebone",2016-12-28 00:00:00,116,"Little Argyll Street, West End"
3,61343325,1140.0,4208.0,2016-12-28 00:20:00,468.0,"Cantrell Road, Bow",2016-12-28 00:01:00,443,"Philpot Street, Whitechapel"
4,61343324,NaN,1406.0,NaN,NaN,NaN,2016-12-28 00:01:00,319,"Baldwin Street, St. Luke's"
38215555,101367955,240.0,15436.0,2020-09-01 23:53:00,652.0,"Evesham Street, Avondale",2020-09-01 23:49:00,606,"Addison Road, Holland Park"
38215556,101367958,60.0,10896.0,2020-09-01 23:51:00,488.0,"Reardon Street, Wapping",2020-09-01 23:50:00,458,"Wapping Lane, Wapping"
38215557,101367970,480.0,18116.0,2020-09-01 23:59:00,442.0,"Walmer Road, Avondale",2020-09-01 23:51:00,442,"Walmer Road, Avondale"
38215558,101367999,300.0,16371.0,2020-09-01 23:59:00,511.0,"Sutton Street, Shadwell",2020-09-01 23:54:00,202,"Leman Street, Aldgate"
38215559,101367995,240.0,8408.0,2020-09-01 23:58:00,56.0,"Paddington Street, Marylebone",2020-09-01 23:54:00,367,"Harrowby Street, Marylebone"


In [21]:
%%time
# pandas chuncks
iterator = pd.read_csv(csvfile, chunksize=chunksize) 
chuncks = []
for chunck in iterator: # time-consuming
    chuncks.append(chunck.groupby(['bike_id'], as_index=False).agg({'duration':'sum','rental_id':'count'}))
df = pd.concat(chuncks) 
# finalize
grouped_df = df.groupby(['bike_id'], as_index=False).agg({'duration':'sum','rental_id':'sum'})

Wall time: 39.8 s


In [22]:
grouped_df.shape

(16019, 3)

In [23]:
pd.concat([grouped_df.head(),grouped_df.tail()])

,bike_id,duration,rental_id
0,1.0,3889465.0,2653
1,2.0,2056260.0,1569
2,4.0,4086285.0,2905
3,5.0,3847900.0,2665
4,6.0,3776526.0,3092
16014,18499.0,6360.0,7
16015,18502.0,6960.0,8
16016,18505.0,4560.0,4
16017,18510.0,19440.0,14
16018,18512.0,8160.0,9


## 3. The `dask` dataframe with `csv` file

In [24]:
%%time
# dask dataframe
# read file
df = None
df = dd.read_csv(csvfile, assume_missing=True)
df = df.compute() # time-consuming

Wall time: 33.9 s


In [25]:
df.shape

(38215560, 9)

In [26]:
pd.concat([df.head(),df.tail()])

,rental_id,duration,bike_id,end_rental_date_time,end_station_id,end_station_name,start_rental_date_time,start_station_id,start_station_name
0,61343322.0,60.0,12871.0,2016-12-28 00:01:00,660.0,"West Kensington Station, West Kensington",2016-12-28 00:00:00,633.0,"Vereker Road North, West Kensington"
1,61343321.0,300.0,2837.0,2016-12-28 00:05:00,763.0,"Mile End Park Leisure Centre, Mile End",2016-12-28 00:00:00,531.0,"Twig Folly Bridge, Mile End"
2,61343323.0,360.0,1269.0,2016-12-28 00:06:00,99.0,"Old Quebec Street, Marylebone",2016-12-28 00:00:00,116.0,"Little Argyll Street, West End"
3,61343325.0,1140.0,4208.0,2016-12-28 00:20:00,468.0,"Cantrell Road, Bow",2016-12-28 00:01:00,443.0,"Philpot Street, Whitechapel"
4,61343324.0,NaN,1406.0,<NA>,NaN,<NA>,2016-12-28 00:01:00,319.0,"Baldwin Street, St. Luke's"
478034,101367955.0,240.0,15436.0,2020-09-01 23:53:00,652.0,"Evesham Street, Avondale",2020-09-01 23:49:00,606.0,"Addison Road, Holland Park"
478035,101367958.0,60.0,10896.0,2020-09-01 23:51:00,488.0,"Reardon Street, Wapping",2020-09-01 23:50:00,458.0,"Wapping Lane, Wapping"
478036,101367970.0,480.0,18116.0,2020-09-01 23:59:00,442.0,"Walmer Road, Avondale",2020-09-01 23:51:00,442.0,"Walmer Road, Avondale"
478037,101367999.0,300.0,16371.0,2020-09-01 23:59:00,511.0,"Sutton Street, Shadwell",2020-09-01 23:54:00,202.0,"Leman Street, Aldgate"
478038,101367995.0,240.0,8408.0,2020-09-01 23:58:00,56.0,"Paddington Street, Marylebone",2020-09-01 23:54:00,367.0,"Harrowby Street, Marylebone"


In [27]:
%%time
# dask dataframe
# group by
df = None
df = dd.read_csv(csvfile, assume_missing=True)
grouped_df = df.groupby(['bike_id']).agg({'duration':'sum','rental_id':'count'})
grouped_df = grouped_df.compute() # time-consuming
grouped_df = grouped_df.sort_index().reset_index()

Wall time: 33.2 s


In [28]:
grouped_df.shape

(16019, 3)

In [29]:
pd.concat([grouped_df.head(),grouped_df.tail()])

,bike_id,duration,rental_id
0,1.0,3889465.0,2653
1,2.0,2056260.0,1569
2,4.0,4086285.0,2905
3,5.0,3847900.0,2665
4,6.0,3776526.0,3092
16014,18499.0,6360.0,7
16015,18502.0,6960.0,8
16016,18505.0,4560.0,4
16017,18510.0,19440.0,14
16018,18512.0,8160.0,9


## 4. The `pandas` on `ray` (`modin`) with `csv` file

In [30]:
ray.shutdown()
ray.init(num_cpus=num_cpus, 
         ignore_reinit_error=True)

2023-09-15 13:48:08,085	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.12
Ray version:,2.2.0
Dashboard:,http://127.0.0.1:8265


In [31]:
os.environ['__MODIN_AUTOIMPORT_PANDAS__'] = '1'

In [32]:
os.environ['MODIN_ENGINE'] = 'ray'

In [33]:
%%time
# modin pandas
# read file
df = None
df = md.read_csv(csvfile) # time-consuming

Wall time: 37.9 s


In [34]:
df.shape

(38215560, 9)

In [35]:
md.concat([df.head(),df.tail()])

,rental_id,duration,bike_id,end_rental_date_time,end_station_id,end_station_name,start_rental_date_time,start_station_id,start_station_name
0,61343322,60.0,12871.0,2016-12-28 00:01:00,660.0,"West Kensington Station, West Kensington",2016-12-28 00:00:00,633,"Vereker Road North, West Kensington"
1,61343321,300.0,2837.0,2016-12-28 00:05:00,763.0,"Mile End Park Leisure Centre, Mile End",2016-12-28 00:00:00,531,"Twig Folly Bridge, Mile End"
2,61343323,360.0,1269.0,2016-12-28 00:06:00,99.0,"Old Quebec Street, Marylebone",2016-12-28 00:00:00,116,"Little Argyll Street, West End"
3,61343325,1140.0,4208.0,2016-12-28 00:20:00,468.0,"Cantrell Road, Bow",2016-12-28 00:01:00,443,"Philpot Street, Whitechapel"
4,61343324,NaN,1406.0,NaN,NaN,NaN,2016-12-28 00:01:00,319,"Baldwin Street, St. Luke's"
38215555,101367955,240.0,15436.0,2020-09-01 23:53:00,652.0,"Evesham Street, Avondale",2020-09-01 23:49:00,606,"Addison Road, Holland Park"
38215556,101367958,60.0,10896.0,2020-09-01 23:51:00,488.0,"Reardon Street, Wapping",2020-09-01 23:50:00,458,"Wapping Lane, Wapping"
38215557,101367970,480.0,18116.0,2020-09-01 23:59:00,442.0,"Walmer Road, Avondale",2020-09-01 23:51:00,442,"Walmer Road, Avondale"
38215558,101367999,300.0,16371.0,2020-09-01 23:59:00,511.0,"Sutton Street, Shadwell",2020-09-01 23:54:00,202,"Leman Street, Aldgate"
38215559,101367995,240.0,8408.0,2020-09-01 23:58:00,56.0,"Paddington Street, Marylebone",2020-09-01 23:54:00,367,"Harrowby Street, Marylebone"


In [36]:
%%time
# modin pandas
# group by
df = None
df = md.read_csv(csvfile) # time-consuming
grouped_df = df.groupby(['bike_id']).agg({'duration':'sum','rental_id':'count'})
grouped_df = grouped_df.reset_index()

Wall time: 36.2 s


In [37]:
grouped_df.shape

(16019, 3)

In [38]:
md.concat([grouped_df.head(),grouped_df.tail()])

,bike_id,duration,rental_id
0,1.0,3889465.0,2653
1,2.0,2056260.0,1569
2,4.0,4086285.0,2905
3,5.0,3847900.0,2665
4,6.0,3776526.0,3092
16014,18499.0,6360.0,7
16015,18502.0,6960.0,8
16016,18505.0,4560.0,4
16017,18510.0,19440.0,14
16018,18512.0,8160.0,9


In [39]:
ray.shutdown()

## 5. The `pyspark` with `csv` file

In [40]:
df.columns

Index(['rental_id', 'duration', 'bike_id', 'end_rental_date_time',
       'end_station_id', 'end_station_name', 'start_rental_date_time',
       'start_station_id', 'start_station_name'],
      dtype='object')

In [41]:
%%time
spark = SparkSession.builder.appName('readingLargeCSVFile')\
                            .getOrCreate()

schema = StructType() \
        .add('rental_id',IntegerType(),True) \
        .add('duration' ,FloatType(),True) \
        .add('bike_id'  ,FloatType(),True) \
        .add('end_rental_date_time' ,TimestampType(),True) \
        .add('end_station_id' ,FloatType(),True) \
        .add('end_station_name' ,StringType(),True) \
        .add('start_rental_date_time' ,TimestampType(),True) \
        .add('start_station_id' ,IntegerType(),True) \
        .add('start_station_name' ,StringType(),True) \

df = spark.read.format('csv').options(header='True', inferSchema='False', delimiter=',').schema(schema).load(csvfile) 

df.printSchema()
df.show(10)

spark.stop()

root
 |-- rental_id: integer (nullable = true)
 |-- duration: float (nullable = true)
 |-- bike_id: float (nullable = true)
 |-- end_rental_date_time: timestamp (nullable = true)
 |-- end_station_id: float (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- start_rental_date_time: timestamp (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)

+---------+--------+-------+--------------------+--------------+--------------------+----------------------+----------------+--------------------+
|rental_id|duration|bike_id|end_rental_date_time|end_station_id|    end_station_name|start_rental_date_time|start_station_id|  start_station_name|
+---------+--------+-------+--------------------+--------------+--------------------+----------------------+----------------+--------------------+
| 61343322|    60.0|12871.0| 2016-12-28 00:01:00|         660.0|West Kensington S...|   2016-12-28 00:00:00|             633|Verek

In [42]:
%%time
spark = SparkSession.builder.appName('readingLargeCSVFile')\
                            .getOrCreate()

schema = StructType() \
        .add('rental_id',IntegerType(),True) \
        .add('duration' ,FloatType(),True) \
        .add('bike_id'  ,FloatType(),True) \
        .add('end_rental_date_time' ,TimestampType(),True) \
        .add('end_station_id' ,FloatType(),True) \
        .add('end_station_name' ,StringType(),True) \
        .add('start_rental_date_time' ,TimestampType(),True) \
        .add('start_station_id' ,IntegerType(),True) \
        .add('start_station_name' ,StringType(),True) \

df = spark.read.format('csv').options(header='True', inferSchema='False', delimiter=',').schema(schema).load(csvfile) 

grouped_df = df.groupBy(['bike_id']).agg({'duration':'sum','rental_id':'count'})
grouped_df = grouped_df.sort(PysparkFunc.col('bike_id').asc())

grouped_df.printSchema()
grouped_df.show(10)

spark.stop()

root
 |-- bike_id: float (nullable = true)
 |-- sum(duration): double (nullable = true)
 |-- count(rental_id): long (nullable = false)

+-------+-------------+----------------+
|bike_id|sum(duration)|count(rental_id)|
+-------+-------------+----------------+
|   null|     267180.0|              17|
|    1.0|    3889465.0|            2653|
|    2.0|    2056260.0|            1569|
|    4.0|    4086285.0|            2905|
|    5.0|    3847900.0|            2665|
|    6.0|    3776526.0|            3092|
|    7.0|    1360755.0|            1188|
|    8.0|    4212032.0|            3233|
|    9.0|    4161402.0|            3112|
|   10.0|    2803224.0|            2309|
+-------+-------------+----------------+
only showing top 10 rows

Wall time: 15.8 s


## 6. Traditional `pandas` with `parquet` file

In [8]:
parquetfile = data_path+'bikes-1.parquet'

In [45]:
%%time
# dask dataframe
# create parquet file
df = None
df = dd.read_csv(csvfile, assume_missing=True)
df.to_parquet(parquetfile)

Wall time: 39.1 s


In [46]:
%%time
# traditional pandas
# read file
df = None
df = pd.read_parquet(parquetfile)

Wall time: 1min


In [47]:
df.shape

(38215560, 9)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38215560 entries, 0 to 485435
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   rental_id               float64
 1   duration                float64
 2   bike_id                 float64
 3   end_rental_date_time    string 
 4   end_station_id          float64
 5   end_station_name        string 
 6   start_rental_date_time  string 
 7   start_station_id        float64
 8   start_station_name      string 
dtypes: float64(5), string(4)
memory usage: 2.8 GB


In [49]:
df.isnull().sum()

rental_id                     0
duration                  67617
bike_id                      17
end_rental_date_time      67617
end_station_id            68213
end_station_name          68213
start_rental_date_time        0
start_station_id              0
start_station_name            0
dtype: int64

In [50]:
df['rental_id'].is_unique

True

In [51]:
pd.concat([df.head(),df.tail()])

,rental_id,duration,bike_id,end_rental_date_time,end_station_id,end_station_name,start_rental_date_time,start_station_id,start_station_name
__null_dask_index__,,,,,,,,,
0,61343322.0,60.0,12871.0,2016-12-28 00:01:00,660.0,"West Kensington Station, West Kensington",2016-12-28 00:00:00,633.0,"Vereker Road North, West Kensington"
1,61343321.0,300.0,2837.0,2016-12-28 00:05:00,763.0,"Mile End Park Leisure Centre, Mile End",2016-12-28 00:00:00,531.0,"Twig Folly Bridge, Mile End"
2,61343323.0,360.0,1269.0,2016-12-28 00:06:00,99.0,"Old Quebec Street, Marylebone",2016-12-28 00:00:00,116.0,"Little Argyll Street, West End"
3,61343325.0,1140.0,4208.0,2016-12-28 00:20:00,468.0,"Cantrell Road, Bow",2016-12-28 00:01:00,443.0,"Philpot Street, Whitechapel"
4,61343324.0,NaN,1406.0,<NA>,NaN,<NA>,2016-12-28 00:01:00,319.0,"Baldwin Street, St. Luke's"
485431,66468067.0,840.0,4881.0,2017-06-23 07:39:00,211.0,"Cadogan Place, Knightsbridge",2017-06-23 07:25:00,45.0,"Boston Place, Marylebone"
485432,66468068.0,420.0,5930.0,2017-06-23 07:32:00,14.0,"Belgrove Street , King's Cross",2017-06-23 07:25:00,81.0,"Great Titchfield Street, Fitzrovia"
485433,66468061.0,300.0,3881.0,2017-06-23 07:30:00,521.0,"Driffield Road, Old Ford",2017-06-23 07:25:00,522.0,"Clinton Road, Mile End"
485434,66468053.0,660.0,11979.0,2017-06-23 07:36:00,387.0,"Fire Brigade Pier, Vauxhall",2017-06-23 07:25:00,354.0,"Northumberland Avenue, Strand"


In [52]:
%%time
# traditional pandas
# group by
df = None
df = pd.read_parquet(parquetfile)
grouped_df = df.groupby(['bike_id'], as_index=False).agg({'duration':'sum','rental_id':'count'})

Wall time: 1min 18s


In [53]:
grouped_df.shape

(16019, 3)

In [54]:
pd.concat([grouped_df.head(),grouped_df.tail()])

,bike_id,duration,rental_id
0,1.0,3889465.0,2653
1,2.0,2056260.0,1569
2,4.0,4086285.0,2905
3,5.0,3847900.0,2665
4,6.0,3776526.0,3092
16014,18499.0,6360.0,7
16015,18502.0,6960.0,8
16016,18505.0,4560.0,4
16017,18510.0,19440.0,14
16018,18512.0,8160.0,9


## 7. The `dask` dataframe with `parquet` file

In [55]:
%%time
# dask dataframe
# read file
df = None
df = dd.read_parquet(parquetfile)
df = df.compute() # time-consuming

Wall time: 18.6 s


In [56]:
df.shape

(38215560, 9)

In [57]:
pd.concat([df.head(),df.tail()])

,rental_id,duration,bike_id,end_rental_date_time,end_station_id,end_station_name,start_rental_date_time,start_station_id,start_station_name
0,61343322.0,60.0,12871.0,2016-12-28 00:01:00,660.0,"West Kensington Station, West Kensington",2016-12-28 00:00:00,633.0,"Vereker Road North, West Kensington"
1,61343321.0,300.0,2837.0,2016-12-28 00:05:00,763.0,"Mile End Park Leisure Centre, Mile End",2016-12-28 00:00:00,531.0,"Twig Folly Bridge, Mile End"
2,61343323.0,360.0,1269.0,2016-12-28 00:06:00,99.0,"Old Quebec Street, Marylebone",2016-12-28 00:00:00,116.0,"Little Argyll Street, West End"
3,61343325.0,1140.0,4208.0,2016-12-28 00:20:00,468.0,"Cantrell Road, Bow",2016-12-28 00:01:00,443.0,"Philpot Street, Whitechapel"
4,61343324.0,NaN,1406.0,<NA>,NaN,<NA>,2016-12-28 00:01:00,319.0,"Baldwin Street, St. Luke's"
478034,101367955.0,240.0,15436.0,2020-09-01 23:53:00,652.0,"Evesham Street, Avondale",2020-09-01 23:49:00,606.0,"Addison Road, Holland Park"
478035,101367958.0,60.0,10896.0,2020-09-01 23:51:00,488.0,"Reardon Street, Wapping",2020-09-01 23:50:00,458.0,"Wapping Lane, Wapping"
478036,101367970.0,480.0,18116.0,2020-09-01 23:59:00,442.0,"Walmer Road, Avondale",2020-09-01 23:51:00,442.0,"Walmer Road, Avondale"
478037,101367999.0,300.0,16371.0,2020-09-01 23:59:00,511.0,"Sutton Street, Shadwell",2020-09-01 23:54:00,202.0,"Leman Street, Aldgate"
478038,101367995.0,240.0,8408.0,2020-09-01 23:58:00,56.0,"Paddington Street, Marylebone",2020-09-01 23:54:00,367.0,"Harrowby Street, Marylebone"


In [58]:
%%time
# dask dataframe
# group by
df = None
df = dd.read_parquet(parquetfile)
grouped_df = df.groupby(['bike_id']).agg({'duration':'sum','rental_id':'count'})
grouped_df = grouped_df.compute() # time-consuming
grouped_df = grouped_df.sort_index().reset_index()

Wall time: 2.54 s


In [59]:
grouped_df.shape

(16019, 3)

In [60]:
pd.concat([grouped_df.head(),grouped_df.tail()])

,bike_id,duration,rental_id
0,1.0,3889465.0,2653
1,2.0,2056260.0,1569
2,4.0,4086285.0,2905
3,5.0,3847900.0,2665
4,6.0,3776526.0,3092
16014,18499.0,6360.0,7
16015,18502.0,6960.0,8
16016,18505.0,4560.0,4
16017,18510.0,19440.0,14
16018,18512.0,8160.0,9


## 8. The `pandas` on `ray` (`modin`) with `parquet` file

In [9]:
ray.shutdown()
ray.init(num_cpus=num_cpus, 
         ignore_reinit_error=True)

2023-09-15 13:59:58,206	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.12
Ray version:,2.2.0
Dashboard:,http://127.0.0.1:8265


In [10]:
os.environ['__MODIN_AUTOIMPORT_PANDAS__'] = '1'

In [11]:
os.environ['MODIN_ENGINE'] = 'ray'

In [12]:
%%time
# modin pandas
# read file
df = None
df = md.read_parquet(parquetfile) # time-consuming

Wall time: 1min 6s


In [13]:
df.shape

(38215560, 9)

In [14]:
md.concat([df.head(),df.tail()])

,rental_id,duration,bike_id,end_rental_date_time,end_station_id,end_station_name,start_rental_date_time,start_station_id,start_station_name
__null_dask_index__,,,,,,,,,
0,61343322.0,60.0,12871.0,2016-12-28 00:01:00,660.0,"West Kensington Station, West Kensington",2016-12-28 00:00:00,633.0,"Vereker Road North, West Kensington"
1,61343321.0,300.0,2837.0,2016-12-28 00:05:00,763.0,"Mile End Park Leisure Centre, Mile End",2016-12-28 00:00:00,531.0,"Twig Folly Bridge, Mile End"
2,61343323.0,360.0,1269.0,2016-12-28 00:06:00,99.0,"Old Quebec Street, Marylebone",2016-12-28 00:00:00,116.0,"Little Argyll Street, West End"
3,61343325.0,1140.0,4208.0,2016-12-28 00:20:00,468.0,"Cantrell Road, Bow",2016-12-28 00:01:00,443.0,"Philpot Street, Whitechapel"
4,61343324.0,NaN,1406.0,<NA>,NaN,<NA>,2016-12-28 00:01:00,319.0,"Baldwin Street, St. Luke's"
485431,66468067.0,840.0,4881.0,2017-06-23 07:39:00,211.0,"Cadogan Place, Knightsbridge",2017-06-23 07:25:00,45.0,"Boston Place, Marylebone"
485432,66468068.0,420.0,5930.0,2017-06-23 07:32:00,14.0,"Belgrove Street , King's Cross",2017-06-23 07:25:00,81.0,"Great Titchfield Street, Fitzrovia"
485433,66468061.0,300.0,3881.0,2017-06-23 07:30:00,521.0,"Driffield Road, Old Ford",2017-06-23 07:25:00,522.0,"Clinton Road, Mile End"
485434,66468053.0,660.0,11979.0,2017-06-23 07:36:00,387.0,"Fire Brigade Pier, Vauxhall",2017-06-23 07:25:00,354.0,"Northumberland Avenue, Strand"


In [15]:
%%time
# modin pandas
# group by
df = None
df = md.read_parquet(parquetfile) # time-consuming
grouped_df = df.groupby(['bike_id']).agg({'duration':'sum','rental_id':'count'})
grouped_df = grouped_df.reset_index()

Wall time: 1min 4s


In [16]:
grouped_df.shape

(16019, 3)

In [17]:
md.concat([grouped_df.head(),grouped_df.tail()])

,bike_id,rental_id,duration
0,1.0,2653,3889465.0
1,2.0,1569,2056260.0
2,4.0,2905,4086285.0
3,5.0,2665,3847900.0
4,6.0,3092,3776526.0
16014,18499.0,7,6360.0
16015,18502.0,8,6960.0
16016,18505.0,4,4560.0
16017,18510.0,14,19440.0
16018,18512.0,9,8160.0


In [18]:
ray.shutdown()

## 9. The `pyspark` with `parquet` file

In [19]:
df.columns

Index(['rental_id', 'duration', 'bike_id', 'end_rental_date_time',
       'end_station_id', 'end_station_name', 'start_rental_date_time',
       'start_station_id', 'start_station_name'],
      dtype='object')

In [20]:
%%time
spark = SparkSession.builder.appName('readingLargeParquetFile')\
                            .getOrCreate()

df = spark.read.format('parquet').options(header='True', inferSchema='True').load(parquetfile) 

df.printSchema()
df.show(10)

spark.stop()

root
 |-- rental_id: double (nullable = true)
 |-- duration: double (nullable = true)
 |-- bike_id: double (nullable = true)
 |-- end_rental_date_time: string (nullable = true)
 |-- end_station_id: double (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- start_rental_date_time: string (nullable = true)
 |-- start_station_id: double (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- __null_dask_index__: long (nullable = true)

+-----------+--------+-------+--------------------+--------------+--------------------+----------------------+----------------+--------------------+-------------------+
|  rental_id|duration|bike_id|end_rental_date_time|end_station_id|    end_station_name|start_rental_date_time|start_station_id|  start_station_name|__null_dask_index__|
+-----------+--------+-------+--------------------+--------------+--------------------+----------------------+----------------+--------------------+-------------------+
|7.2091291E7|   480.

In [21]:
%%time
spark = SparkSession.builder.appName('readingLargeParquetFile')\
                            .getOrCreate()

df = spark.read.format('parquet').options(header='True', inferSchema='True').load(parquetfile) 
 
grouped_df = df.groupBy(['bike_id']).agg({'duration':'sum','rental_id':'count'})
grouped_df = grouped_df.sort(PysparkFunc.col('bike_id').asc())

grouped_df.printSchema()
grouped_df.show(10)

spark.stop()

root
 |-- bike_id: double (nullable = true)
 |-- sum(duration): double (nullable = true)
 |-- count(rental_id): long (nullable = false)

+-------+-------------+----------------+
|bike_id|sum(duration)|count(rental_id)|
+-------+-------------+----------------+
|   null|     267180.0|              17|
|    1.0|    3889465.0|            2653|
|    2.0|    2056260.0|            1569|
|    4.0|    4086285.0|            2905|
|    5.0|    3847900.0|            2665|
|    6.0|    3776526.0|            3092|
|    7.0|    1360755.0|            1188|
|    8.0|    4212032.0|            3233|
|    9.0|    4161402.0|            3112|
|   10.0|    2803224.0|            2309|
+-------+-------------+----------------+
only showing top 10 rows

Wall time: 5.61 s


## Conclusion

The steps for an efficient loading of a `CSV` file into a dataframe is:

1. converting the `csv` file into a `parquet` format using `dask`.
2. reading and processing the `parquet` file using `pyspark`.